## UCSC Browser
In this tutorial we will use Python MySQL to connect to the UCSC browser database to make our queries. There
are several advantages to using this method one being that you will retrieve the results in your Python workspace.
This avoids the extra step of first downloading the data and then loading it into Python.
Additionally you can also write functions that will allow you to automate certain queries.


## Installing RMySQL
To install MySQL, simply use the standard method.

In [1]:
# !pip install mysql-connector-python

To make sure that the package installed properly, use the import command

In [2]:
import mysql.connector
import pandas as pd

### Connect to ucsc genome mysql database
To connect to the UCSC browser we can use the information provided on the site. https://genome.ucsc.edu/goldenPath/help/mysql.html
#### It states that in order to connect we should use the following credentials.
mysql –user=genome –host=genome-mysql.cse.ucsc.edu -A
### Let’s create a connection to the database.

In [3]:
# connect to ucsc server
ucsccon = mysql.connector.connect(user='genome', 
                        host='genome-mysql.cse.ucsc.edu')

# create a cursor
c = ucsccon.cursor()
c.execute("show databases")

To get a list of all the databases available we need to iterate the cursor

In [4]:
# db list
dblist = []

for databases in c:
     dblist.append(databases[0])

print("Number of DBs",len(dblist), "\n")
print("Header of db list",dblist[0:5], "\n")
print("Elements that contains keyword 'hg':", [i for i in dblist if 'hg' in i])

Number of DBs 335 

Header of db list ['acaChl1', 'ailMel1', 'allMis1', 'allSin1', 'amaVit1'] 

Elements that contains keyword 'hg': ['hg16', 'hg17', 'hg18', 'hg19', 'hg19Patch10', 'hg19Patch13', 'hg38', 'hg38Patch11', 'hgFixed', 'hgcentral']


### Now let’s select the database for further analysis

In [5]:
c.execute("use hg19")
c.execute("show tables")
hg19_list = []
for table in c:
     hg19_list.append(table[0])

In [6]:
hg19_list[0:5]

['HInv', 'HInvGeneMrna', 'acembly', 'acemblyClass', 'acemblyPep']

### Specifically we are interested in finding all tables that contain snp information.

In [7]:
[i for i in hg19_list if "snp" in i]

['snp138',
 'snp138CodingDbSnp',
 'snp138Common',
 'snp138ExceptionDesc',
 'snp138Flagged',
 'snp138Mult',
 'snp138OrthoPt4Pa2Rm3',
 'snp138Seq',
 'snp141',
 'snp141CodingDbSnp',
 'snp141Common',
 'snp141ExceptionDesc',
 'snp141Flagged',
 'snp141OrthoPt4Pa2Rm3',
 'snp141Seq',
 'snp142',
 'snp142CodingDbSnp',
 'snp142Common',
 'snp142ExceptionDesc',
 'snp142Flagged',
 'snp142Mult',
 'snp142OrthoPt4Pa2Rm3',
 'snp142Seq',
 'snp144',
 'snp144CodingDbSnp',
 'snp144Common',
 'snp144ExceptionDesc',
 'snp144Flagged',
 'snp144Mult',
 'snp144OrthoPt4Pa2Rm3',
 'snp144Seq',
 'snp146',
 'snp146CodingDbSnp',
 'snp146Common',
 'snp146ExceptionDesc',
 'snp146Flagged',
 'snp146Mult',
 'snp146OrthoPt4Pa2Rm3',
 'snp146Seq',
 'snp147',
 'snp147CodingDbSnp',
 'snp147Common',
 'snp147ExceptionDesc',
 'snp147Flagged',
 'snp147Mult',
 'snp147OrthoPt4Pa2Rm3',
 'snp147Seq',
 'snp150',
 'snp150CodingDbSnp',
 'snp150Common',
 'snp150ExceptionDesc',
 'snp150Flagged',
 'snp150Mult',
 'snp150OrthoPt5Pa2Rm8',
 'snp15

### To learn about the fields that are in the snp138Common table

In [8]:
c.execute("describe snp138Common")
for i in c:
     print(i[0])

bin
chrom
chromStart
chromEnd
name
score
strand
refNCBI
refUCSC
observed
molType
class
valid
avHet
avHetSE
func
locType
weight
exceptions
submitterCount
submitters
alleleFreqCount
alleles
alleleNs
alleleFreqs
bitfields


Refgenes ( Reference genes ) is a database that contains anotated genes from a genome. We can
look for all the genes in the refGene table to learn and search for genes of interest. Performing
a quick select query on the tables shows us that name is used as a field for ref gene id and
name2 is the field for more common gene names.

In [9]:
pd.read_sql("""
        select name,chrom,txStart,txEnd,name2 
        from refGene limit 10
""", ucsccon)

,name,chrom,txStart,txEnd,name2
0,NR_046630,chr3,196666747,196669405,NCBP2-AS1
1,NR_046598,chr3,192232810,192234362,FGF12-AS2
2,NR_046514,chr13,95364969,95368199,SOX21-AS1
3,NR_106918,chr1,17368,17436,MIR6859-1
4,NR_107062,chr1,17368,17436,MIR6859-2
5,NR_107063,chr1,17368,17436,MIR6859-3
6,NR_128720,chr1,17368,17436,MIR6859-4
7,NR_036051,chr1,30365,30503,MIR1302-2
8,NR_036266,chr1,30365,30503,MIR1302-9
9,NR_036267,chr1,30365,30503,MIR1302-10


#### Let’s find the coordinates for BRCA2

In [10]:
pd.read_sql("""
        select name,chrom,txStart,txEnd,name2
        from refGene where name2=\"BRCA2\"
""", ucsccon)

,name,chrom,txStart,txEnd,name2
0,NM_000059,chr13,32889644,32974405,BRCA2


#### Now using these coordinates we can identify all coding-synonymous snps from the snp138Common table.


In [11]:
pd.read_sql("""
        select chrom, chromStart, ChromEnd, name, func
        from snp138Common where
        chrom=\"chr13\" and
        chromStart < 32973809 and
        chromEnd > 32889616 and
        func like \"%coding-synon%\"
""", ucsccon)

,chrom,chromStart,ChromEnd,name,func
0,chr13,32906898,32906899,rs34770647,{coding-synon}
1,chr13,32906979,32906980,rs1801439,{coding-synon}
2,chr13,32910720,32910721,rs1801499,{coding-synon}
3,chr13,32911887,32911888,rs1801406,"{coding-synon, missense}"
4,chr13,32912298,32912299,rs543304,{coding-synon}
5,chr13,32913054,32913055,rs206075,{coding-synon}
6,chr13,32915004,32915005,rs206076,{coding-synon}
7,chr13,32929231,32929232,rs1799955,{coding-synon}
8,chr13,32944666,32944667,rs9590940,{coding-synon}


#### How can we combine the two queries using join ?

In [12]:
pd.read_sql("""
        select snp138Common.chrom, snp138Common.chromStart,
        snp138Common.ChromEnd, snp138Common.name, snp138Common.func
        from snp138Common,refGene where
        refGene.name2=\"BRCA2\" and
        snp138Common.chrom=refGene.chrom and
        snp138Common.chromStart < refGene.txEnd and
        snp138Common.chromEnd > refGene.txStart and
        snp138Common.func like \"%coding-synon%\"
""", ucsccon)

,chrom,chromStart,ChromEnd,name,func
0,chr13,32906898,32906899,rs34770647,{coding-synon}
1,chr13,32906979,32906980,rs1801439,{coding-synon}
2,chr13,32910720,32910721,rs1801499,{coding-synon}
3,chr13,32911887,32911888,rs1801406,"{coding-synon, missense}"
4,chr13,32912298,32912299,rs543304,{coding-synon}
5,chr13,32913054,32913055,rs206075,{coding-synon}
6,chr13,32915004,32915005,rs206076,{coding-synon}
7,chr13,32929231,32929232,rs1799955,{coding-synon}
8,chr13,32944666,32944667,rs9590940,{coding-synon}


#### How can we provide BRCA2 as a variable name so that we can repeat the query for several different genes ?

In [13]:
gene_query = "\"BRCA2\""
func_query = "\"%coding-synon%\""
tmp_query = """
        select snp138Common.chrom, snp138Common.chromStart, 
        snp138Common.ChromEnd, snp138Common.name, snp138Common.func
        from snp138Common,refGene where
        refGene.name2= %s and
        snp138Common.chrom=refGene.chrom and
        snp138Common.chromStart < refGene.txEnd and
        snp138Common.chromEnd > refGene.txStart and
        snp138Common.func like %s
"""%(gene_query, func_query)

In [14]:
pd.read_sql(tmp_query, ucsccon)

,chrom,chromStart,ChromEnd,name,func
0,chr13,32906898,32906899,rs34770647,{coding-synon}
1,chr13,32906979,32906980,rs1801439,{coding-synon}
2,chr13,32910720,32910721,rs1801499,{coding-synon}
3,chr13,32911887,32911888,rs1801406,"{coding-synon, missense}"
4,chr13,32912298,32912299,rs543304,{coding-synon}
5,chr13,32913054,32913055,rs206075,{coding-synon}
6,chr13,32915004,32915005,rs206076,{coding-synon}
7,chr13,32929231,32929232,rs1799955,{coding-synon}
8,chr13,32944666,32944667,rs9590940,{coding-synon}


In [15]:
gene_query2 = "BRCA2"
func_query2 = "%coding-synon%"
tmp_query2 = """
        select snp138Common.chrom, snp138Common.chromStart,
        snp138Common.ChromEnd, snp138Common.name, snp138Common.func
        from snp138Common,refGene where
        refGene.name2= '""" + gene_query2 + """' and
        snp138Common.chrom=refGene.chrom and
        snp138Common.chromStart < refGene.txEnd and
        snp138Common.chromEnd > refGene.txStart and
        snp138Common.func like '""" + func_query2 +"""'"""

In [16]:
pd.read_sql(tmp_query2, ucsccon)

,chrom,chromStart,ChromEnd,name,func
0,chr13,32906898,32906899,rs34770647,{coding-synon}
1,chr13,32906979,32906980,rs1801439,{coding-synon}
2,chr13,32910720,32910721,rs1801499,{coding-synon}
3,chr13,32911887,32911888,rs1801406,"{coding-synon, missense}"
4,chr13,32912298,32912299,rs543304,{coding-synon}
5,chr13,32913054,32913055,rs206075,{coding-synon}
6,chr13,32915004,32915005,rs206076,{coding-synon}
7,chr13,32929231,32929232,rs1799955,{coding-synon}
8,chr13,32944666,32944667,rs9590940,{coding-synon}


## Note: the code above is subjective to injection attack, please use placeholder %()s
#### rewrite the function using query parameters instead of string interpolation

In [17]:
param = {"gene":"BRCA2",
        "func":"%coding-synon%"}


pd.read_sql_query("""select snp138Common.chrom,
    snp138Common.chromStart,
    snp138Common.ChromEnd,
    snp138Common.name, snp138Common.func
    from snp138Common,refGene where
    refGene.name2= %(gene)s and
    snp138Common.chrom=refGene.chrom and
    snp138Common.chromStart < refGene.txEnd and
    snp138Common.chromEnd > refGene.txStart and
    snp138Common.func like %(func)s """, ucsccon, params= param)

,chrom,chromStart,ChromEnd,name,func
0,chr13,32906898,32906899,rs34770647,{coding-synon}
1,chr13,32906979,32906980,rs1801439,{coding-synon}
2,chr13,32910720,32910721,rs1801499,{coding-synon}
3,chr13,32911887,32911888,rs1801406,"{coding-synon, missense}"
4,chr13,32912298,32912299,rs543304,{coding-synon}
5,chr13,32913054,32913055,rs206075,{coding-synon}
6,chr13,32915004,32915005,rs206076,{coding-synon}
7,chr13,32929231,32929232,rs1799955,{coding-synon}
8,chr13,32944666,32944667,rs9590940,{coding-synon}


### What if have lots of results and instead of getting the results in one show, we prefer to iterate through the one by one.

Use fetchall, fetchmany

In [18]:
c.execute(tmp_query)

In [19]:
# print the first two lines
c.fetchmany(size=2)

[('chr13', 32906898, 32906899, 'rs34770647', {'coding-synon'}),
 ('chr13', 32906979, 32906980, 'rs1801439', {'coding-synon'})]

In [20]:
# print the remaining
c.fetchall()

[('chr13', 32910720, 32910721, 'rs1801499', {'coding-synon'}),
 ('chr13', 32911887, 32911888, 'rs1801406', {'coding-synon', 'missense'}),
 ('chr13', 32912298, 32912299, 'rs543304', {'coding-synon'}),
 ('chr13', 32913054, 32913055, 'rs206075', {'coding-synon'}),
 ('chr13', 32915004, 32915005, 'rs206076', {'coding-synon'}),
 ('chr13', 32929231, 32929232, 'rs1799955', {'coding-synon'}),
 ('chr13', 32944666, 32944667, 'rs9590940', {'coding-synon'})]